In [34]:
import numpy as np
import pandas as pd
# for visualization
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

from sklearn.preprocessing import StandardScaler # for scaling the features.
# Association rules
from mlxtend.frequent_patterns import apriori, association_rules

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree # for decision tree
from sklearn.ensemble import RandomForestClassifier # for random forest
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from sklearn.metrics import roc_curve, auc # plot the ROC curve to compre the classification result

# Heirarchical clustering
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
# Kmeans clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os

# For estimate the clustering results.
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score


In [35]:
#### Functions
# For Standardize
def standardize_features(df, feature_cols):
    """
    Normalize the data.
    """
    scaler = StandardScaler() # normalization setting
    X_scaled = scaler.fit_transform(df[feature_cols].values)
    df_scaled = pd.DataFrame(X_scaled, columns=feature_cols, index=df.index)
    return df_scaled, scaler

# For Association Rule
def discretize_for_association(df, bins = 3):
    """
    Discretize continuous variables into low/medium/high labels
    for association rules. 
    strategy: "uniform" uses cut with equal-width bins.
    Returns a one-hot encoded boolean DataFrame.
    """
    labels = ["low", "medium", "high"][:bins]   # 避免 bins !=3 時報錯
    disc = {}

    for col in df.columns:
        series = df[col]
        try:
            cats = pd.cut(series, bins=bins, labels=labels)
        except ValueError:
            # Fallback: if too few unique values, treat as is
            nbins = min(bins, max(2, series.nunique()))
            fallback_labels = labels[:nbins]
            cats = pd.cut(series, bins=nbins, labels=fallback_labels, duplicates="drop")
        disc[col] = cats.astype(str)

    disc_df = pd.DataFrame(disc, index=df.index)

    # One-hot encoding for mlxtend
    ohe_parts = []
    for col in disc_df.columns:
        dummies = pd.get_dummies(disc_df[col], prefix=col, dtype=bool)
        ohe_parts.append(dummies)

    return pd.concat(ohe_parts, axis=1)


def run_association_rules(ohe_df, min_support = 0.1,
                          min_confidence = 0.5, min_lift = 1.0):
    freq = apriori(ohe_df.astype(bool), min_support=min_support, use_colnames=True)
    if freq.empty:
        return pd.DataFrame()
    rules = association_rules(freq, metric="confidence", min_threshold=min_confidence)
    rules = rules[rules['antecedents'].apply(lambda x: len(x) <= 1)]
    rules = rules[rules['consequents'].apply(lambda x: len(x) == 1)]
    if "lift" in rules.columns:
        rules = rules[rules["lift"] >= min_lift]
    # Sort by confidence then lift
    rules = rules.sort_values(["confidence", "lift"], ascending=False)
    return rules.reset_index(drop=True)

### Data loading

In [36]:
output = "/Users/ching-yalin/Desktop/TIGP/2025_Fall/Biological Computing (C1)/2025-10-27_Introduction to Data Mining (Dr. Henry Horng-Shing Lu)/outputs/"
#### Data loading
wine_df = pd.read_csv("/Users/ching-yalin/Desktop/TIGP/2025_Fall/Biological Computing (C1)/2025-10-27_Introduction to Data Mining (Dr. Henry Horng-Shing Lu)/wine+quality/winequality-red.csv", sep = ';')
#### Standardization
feature_col = [col for col in wine_df if col != 'quality'] # quality is final predicted results
scaled_df, scaler = standardize_features(wine_df, feature_col)
scaled_df.head()
# Note: the scaled data only for PCA and clustering
quality_threshold = 6 # The wine_quality threshold for good/bad label.


### Association Rules

In [37]:
#### Association Rules
df_for_assoc = wine_df.copy()
df_for_assoc['quality_bin'] = np.where(df_for_assoc['quality'] >= quality_threshold,'good','bad')
# Create one-hot encoding table for quality
ohe_quality = pd.get_dummies(df_for_assoc['quality_bin'],prefix='quality',dtype=bool)
# Create one-hot encoding table for other features
ohe_features = discretize_for_association(df_for_assoc[feature_col],bins=3)
# Create one-hot encoding table for association rule analysis
ohe = pd.concat([ohe_features, ohe_quality], axis=1)
rules_all = run_association_rules(ohe, min_support=0.08, min_confidence=0.5, min_lift=1.0)
# Remain the quality association results from all rules
quality_rules = rules_all[rules_all['consequents'].apply(lambda s: ('quality_good' in s) or ('quality_bad' in s))]
# Prevent quality appear in antecedents because we only want to see feature -> quality
quality_rules = quality_rules[~quality_rules['antecedents'].apply(lambda s: any(str(item).startswith('quality_') for item in s))]
quality_rules = quality_rules.reset_index(drop=True)
quality_rules.to_csv( output + "assoc_rules.csv", index=False)
quality_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(alcohol_medium),(quality_good),0.349593,0.534709,0.275172,0.787120,1.472052,1.0,0.088241,2.185694,0.493040,0.451745,0.542479,0.650870
1,(density_low),(quality_good),0.106942,0.534709,0.083177,0.777778,1.454581,1.0,0.025994,2.093809,0.349940,0.148936,0.522401,0.466667
2,(citric acid_medium),(quality_good),0.347092,0.534709,0.227017,0.654054,1.223196,1.0,0.041424,1.344981,0.279472,0.346705,0.256495,0.539308
3,(volatile acidity_medium),(quality_bad),0.308318,0.465291,0.200125,0.649087,1.395014,1.0,0.056668,1.523767,0.409381,0.348964,0.343732,0.539597
4,(alcohol_low),(quality_bad),0.616010,0.465291,0.385241,0.625381,1.344064,1.0,0.098617,1.427340,0.666653,0.553459,0.299396,0.726669


In [38]:
# Support-Confidnece-Lift Scatter plot
plt.figure(figsize=(8,6))
plt.scatter(quality_rules['support'], quality_rules['confidence'],
            c=quality_rules['lift'], cmap='viridis', s=quality_rules['lift']*60)

plt.colorbar(label='Lift')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Association Rules: Support vs Confidence (colored by Lift)')
plt.savefig(os.path.join(output, "assoc_rules_support-confidence-lift.png"), dpi=150)
#plt.show()
plt.close()

In [39]:
## Top 5 rules
quality_rules.sort_values("lift", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(alcohol_medium),(quality_good),0.349593,0.534709,0.275172,0.787120,1.472052,1.0,0.088241,2.185694,0.493040,0.451745,0.542479,0.650870
1,(density_low),(quality_good),0.106942,0.534709,0.083177,0.777778,1.454581,1.0,0.025994,2.093809,0.349940,0.148936,0.522401,0.466667
3,(volatile acidity_medium),(quality_bad),0.308318,0.465291,0.200125,0.649087,1.395014,1.0,0.056668,1.523767,0.409381,0.348964,0.343732,0.539597
4,(alcohol_low),(quality_bad),0.616010,0.465291,0.385241,0.625381,1.344064,1.0,0.098617,1.427340,0.666653,0.553459,0.299396,0.726669
2,(citric acid_medium),(quality_good),0.347092,0.534709,0.227017,0.654054,1.223196,1.0,0.041424,1.344981,0.279472,0.346705,0.256495,0.539308


In [40]:
# Graph network
G = nx.DiGraph()

for _, row in quality_rules.iterrows():
    for a in row['antecedents']:
        for c in row['consequents']:
            G.add_edge(a, c, weight=row['lift'])

plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, k=0.9)
nx.draw(G, pos, with_labels=True, node_size=2200,
        node_color='skyblue', arrowsize=20, font_size=10)
plt.title("Association Rules Network Graph")
plt.savefig(os.path.join(output, "assoc_rules_network_graph.png"), dpi=150)
#plt.show()
plt.close()

In [41]:
# Lift Heatmap
pivot = quality_rules.pivot_table(
    values='lift',
    index=quality_rules['antecedents'].astype(str),
    columns=quality_rules['consequents'].astype(str)
)

plt.figure(figsize=(12, 10))
sns.heatmap(pivot, annot=False, cmap='coolwarm')
plt.title("Lift Heatmap")
plt.savefig(os.path.join(output, "assoc_rules_lift heatmap.png"), dpi=150)
#plt.show()
plt.close()

-------
## Classification Models

In [42]:
# create training data and testing data
df_cls = wine_df.copy() # Use original data (not scaled)
df_cls["high_quality"] = (df_cls["quality"] >= quality_threshold).astype(int) # Turn Ture to 1, False to 0
target_col = "high_quality"


x = df_cls[feature_col].values
y = df_cls[target_col].values

test_size = 0.2
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=42, stratify=y
)

### Decision tree

In [43]:
def run_decision_tree(x_train, x_test, y_train, y_test ,test_size, random_state = 42, max_depth = None):
    # Create the decision tree classifier and apply to the traning data.
    clf = DecisionTreeClassifier(random_state=random_state, max_depth=max_depth, criterion="entropy") # Here the important of feature using entropy-based information gain
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0) # create the performance report
    cm = confusion_matrix(y_test, y_pred) # get confusion matrix

    results = {
        "classification_report": report,
        "confusion_matrix": cm,
        "feature_importances": clf.feature_importances_,
        "test_size": test_size,
    }
    return clf, results


#### Decision tree
clf, dt_results = run_decision_tree(x_train, x_test, y_train, y_test,
                                        test_size=0.2,
                                        random_state=42,
                                        max_depth=None)
# Save report
rep = dt_results["classification_report"]
cm = dt_results["confusion_matrix"]
fi = dt_results["feature_importances"]

with open(os.path.join(output, "decision_tree_report.txt"), "w", encoding="utf-8") as f:
    f.write("=== Classification Report ===\n")
    f.write(pd.DataFrame(rep).T.round(3).to_string())
    f.write("\n\n=== Confusion Matrix ===\n")
    f.write(pd.DataFrame(cm).to_string())
    f.write("\n\n=== Feature Importances ===\n")
    f.write(pd.Series(fi, index=feature_col).sort_values(ascending=False).round(4).to_string())

# Save a quick PNG plot of the tree
plt.figure(figsize=(16, 8))
plot_tree(clf, feature_names=feature_col, class_names=["low", "high"], filled=True, max_depth=3, fontsize=8)
plt.tight_layout()
#plt.show()
plt.savefig(os.path.join(output, "decision_tree.png"), dpi=150)
plt.close()

### Random Forest

In [44]:
def run_random_forest(x_train, x_test, y_train, y_test, test_size,
                      random_state=42, n_estimators=200, max_depth=None):

    # Build Random Forest classifier
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state, criterion="entropy"
    )
    rf.fit(x_train, y_train)

    # Predictions
    y_pred = rf.predict(x_test)

    # Metrics
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    results = {
        "classification_report": report,
        "confusion_matrix": cm,
        "feature_importances": rf.feature_importances_,
        "test_size": test_size
    }
    return rf, results

#### Random Forest
rf_clf, rf_results = run_random_forest(
    x_train, x_test, y_train, y_test,
    test_size=0.2,
    random_state=42,
    n_estimators=200,
    max_depth=None
)

# Save results
rf_rep = rf_results["classification_report"]
rf_cm = rf_results["confusion_matrix"]
rf_fi = rf_results["feature_importances"]

with open(os.path.join(output, "random_forest_report.txt"), "w", encoding="utf-8") as f:
    f.write("=== Classification Report ===\n")
    f.write(pd.DataFrame(rf_rep).T.round(3).to_string())
    f.write("\n\n=== Confusion Matrix ===\n")
    f.write(pd.DataFrame(rf_cm).to_string())
    f.write("\n\n=== Feature Importances ===\n")
    f.write(pd.Series(rf_fi, index=feature_col).sort_values(ascending=False).round(4).to_string())

In [45]:
#### Compare two classification model
# Decision Tree
dt_probs = clf.predict_proba(x_test)[:, 1]   # 機率分數（class 1）
dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_probs)
dt_auc = auc(dt_fpr, dt_tpr)

# Random Forest
rf_probs = rf_clf.predict_proba(x_test)[:, 1]
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probs)
rf_auc = auc(rf_fpr, rf_tpr)

# ROC curve
plt.figure(figsize=(8, 6))
plt.plot(dt_fpr, dt_tpr, label=f"Decision Tree (AUC = {dt_auc:.3f})")
plt.plot(rf_fpr, rf_tpr, label=f"Random Forest (AUC = {rf_auc:.3f})")
plt.plot([0, 1], [0, 1], "k--", label="Random Guess")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison: Decision Tree vs Random Forest")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output, "roc_dt_vs_rf.png"), dpi=150)
#plt.show()
plt.close()

---------------------
## Dimensional reduction

### PCA

In [46]:

# PCA (2 components)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaled_df)

pc1_var = pca.explained_variance_ratio_[0] * 100
pc2_var = pca.explained_variance_ratio_[1] * 100

# Plot
plt.figure(figsize=(8, 6))

scatter = plt.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=y,
    cmap="coolwarm",
    alpha=0.7)

plt.xlabel(f"PC1 ({pc1_var:.2f}% variance)")
plt.ylabel(f"PC2 ({pc2_var:.2f}% variance)")
plt.title("PCA of Wine Quality Dataset")
plt.legend(handles=scatter.legend_elements()[0], labels=["Low Quality", "High Quality"])

plt.tight_layout()
plt.savefig(os.path.join(output, "pca_plot_2d.png"), dpi=150)
plt.close()

In [47]:
### Plot 3D PCA
# PCA with 3 components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(scaled_df)

pc1_var = pca.explained_variance_ratio_[0] * 100
pc2_var = pca.explained_variance_ratio_[1] * 100
pc3_var = pca.explained_variance_ratio_[2] * 100

# Angles
angles = [
    (30, 60)
]

for idx, (elev, azim) in enumerate(angles):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")

    scatter = ax.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        X_pca[:, 2],
        c=y,
        cmap="coolwarm",
        alpha=0.7
    )

    ax.set_xlabel(f"PC1 ({pc1_var:.2f}% variance)")
    ax.set_ylabel(f"PC2 ({pc2_var:.2f}% variance)")
    ax.set_zlabel(f"PC3 ({pc3_var:.2f}% variance)")
    ax.set_title(f"3D PCA View {idx+1} (elev={elev}, azim={azim})")

    legend_elements = scatter.legend_elements()[0]
    ax.legend(legend_elements, ["Low Quality", "High Quality"])

    # Set the viewing angle
    ax.view_init(elev=elev, azim=azim)

    plt.tight_layout()
    save_path = os.path.join(output, f"pca_3d_view_{idx+1}.png")
    plt.savefig(save_path, dpi=150)
    plt.close()


-------------------
## Clustering
### Hierarchical clustering

In [48]:
linkage_method = "ward"
Z = linkage(scaled_df, method="ward")
plt.figure(figsize=(10, 6))
dendrogram(Z,truncate_mode="level", p=5) #plot dendrogram only show 5-level
plt.title(f"Hierarchical Clustering Dendrogram ({linkage_method})")
plt.xlabel("Sample index or (cluster size)")
plt.ylabel("Distance")
plt.tight_layout()
plt.savefig(os.path.join(output, "hierarchi_plot.png"), dpi=150)
#plt.show()
plt.close()

hc = AgglomerativeClustering(
    n_clusters=2,
    linkage=linkage_method,  # "ward"
    metric="euclidean"
)
cluster_labels = hc.fit_predict(scaled_df)

# store the result 
df_cls["hc_cluster"] = cluster_labels

In [ ]:
# Plot the clustering result on PCA
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=df_cls["hc_cluster"],
    cmap="coolwarm",
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA with Hierarchical Clustering Labels")
plt.legend(handles=scatter.legend_elements()[0], labels=[f"Cluster {i}" for i in sorted(df_cls["hc_cluster"].unique())])
plt.tight_layout()
plt.savefig(os.path.join(output, "hierarchi_pca.png"), dpi=150)
#plt.show()
plt.close()

In [50]:
# Estimate the clustering results
true_labels = df_cls["high_quality"]
cluster_labels = df_cls["hc_cluster"]

# Adjusted Rand Index
ari = adjusted_rand_score(true_labels, cluster_labels)

# Normalized Mutual Information
nmi = normalized_mutual_info_score(true_labels, cluster_labels)

# Purity
crosstab = pd.crosstab(cluster_labels, true_labels)
purity = np.sum(crosstab.max(axis=1)) / np.sum(crosstab.values)

print("ARI:", ari)
print("NMI:", nmi)
print("Purity:", purity)

# Plot confusion matrix-like heatmap
cm = pd.crosstab(cluster_labels, true_labels)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

plt.title("Cluster vs True Label")
plt.xlabel("True Label (High Quality 0/1)")
plt.ylabel("Cluster Label (HC Cluster)")
plt.tight_layout()
plt.savefig(os.path.join(output, "hierarchi_heatmap.png"), dpi=200)
#plt.show()
plt.close()

ARI: 0.058596317969862366
NMI: 0.07017058115825778
Purity: 0.6222639149468417


### Clustering (Kmeans)

In [51]:
def run_kmeans(df_scaled, k=2, random_state=42):
    km = KMeans(n_clusters=k, n_init="auto", random_state=random_state)
    labels = km.fit_predict(df_scaled.values)

    sil = np.nan
    if k > 1 and len(df_scaled) > k:
        try:
            sil = silhouette_score(df_scaled.values, labels)
        except Exception:
            sil = np.nan

    labels_series = pd.Series(labels, index=df_scaled.index, name="cluster")
    return km, labels_series, sil


In [52]:
# Using silhouette score to decide K
silhouette_results = []

for k in range(1, 6):
    _, labels_k, sil = run_kmeans(scaled_df, k=k, random_state=42)
    silhouette_results.append({"k": k, "silhouette": sil})

silhouette_df = pd.DataFrame(silhouette_results)

# plot the curve of the silhouette score
plt.figure(figsize=(6, 4))
mask = silhouette_df["k"] > 1
plt.plot(
    silhouette_df.loc[mask, "k"],
    silhouette_df.loc[mask, "silhouette"],
    marker="o"
)
plt.xlabel("Number of clusters (k)")
plt.ylabel("Silhouette score")
plt.title("Silhouette score for K-means (k = 2–5)")
plt.xticks([2, 3, 4, 5])
plt.tight_layout()
plt.savefig(os.path.join(output, "kmeans_silhouette_curve.png"), dpi=150)
#plt.show()
plt.close()

In [ ]:
# Final results
km, labels, silhouette = run_kmeans(scaled_df, k=2, random_state=42)
df_cls["kmeans_cluster"] = labels
df_cls["kmeans_cluster"] = 1 - df_cls["kmeans_cluster"] # because the color is different from heirarchical results, so I reverse it.
# Plot cluster result on PCA
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=df_cls["kmeans_cluster"],
    cmap="coolwarm",
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA with K-means Clustering Labels")
plt.legend(handles=scatter.legend_elements()[0], labels=[f"Cluster {i}" for i in sorted(df_cls["hc_cluster"].unique())])
plt.tight_layout()
plt.savefig(os.path.join(output, "kmeans_pca.png"), dpi=150)
plt.close()



In [ ]:
true_labels = df_cls["high_quality"]
cluster_labels = df_cls["kmeans_cluster"]

# Adjusted Rand Index
ari = adjusted_rand_score(true_labels, cluster_labels)

# Normalized Mutual Information
nmi = normalized_mutual_info_score(true_labels, cluster_labels)

# Purity
crosstab = pd.crosstab(cluster_labels, true_labels)
purity = np.sum(crosstab.max(axis=1)) / np.sum(crosstab.values)

print("ARI:", ari)
print("NMI:", nmi)
print("Purity:", purity)

# Plot confusion matrix-like heatmap
cm = pd.crosstab(cluster_labels, true_labels)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

plt.title("Cluster vs True Label")
plt.xlabel("True Label (High Quality 0/1)")
plt.ylabel("Cluster Label (HC Cluster)")
plt.tight_layout()
plt.savefig(os.path.join(output, "kmeans_heatmap.png"), dpi=200)
#plt.show()
plt.close()

ARI: 0.029125404082489644
NMI: 0.029720560963749056
Purity: 0.5866166353971232
